In [4]:
import numpy as np
import pylab as pl
import scipy as sp
import json
import csv
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# Input your own file path to https://github.com/vanferdi/resource-rational-prediction/blob/main/Data/experiment.csv
file_path = 'experiment.csv'

Inputs = []
Guesses = []
ParticipantIDs = []
Input_Types = []
Hidden_states = []
Accuracy = []
Bonus_round = []

with open(file_path) as csvfile:
		spamreader = csv.reader(csvfile)
		for row in spamreader:
			input_ = row[7]
			guess_ = row[6]
			ID = row[0]
			hmm_states = row[13]
			accuracy = row[22]
			bonus_round = row[29]
			Inputs.append(input_)
			Guesses.append(guess_)
			ParticipantIDs.append(ID)
			Input_Types.append(row[15])
			Hidden_states.append(hmm_states)
			Accuracy.append(accuracy)
			Bonus_round.append(bonus_round)
		Inputs.pop(0)
		Guesses.pop(0)
		ParticipantIDs.pop(0)
		Input_Types.pop(0)
		Hidden_states.pop(0)
		Accuracy.pop(0)
		Bonus_round.pop(0)

with open(file_path) as csvfile:
		spamreader = csv.reader(csvfile)
		for row in spamreader:
			input_ = row[7]
			guess_ = row[6]
			ID = row[0]
			hmm_states = row[13]
			accuracy = row[22]
			bonus_round = row[29]
			Inputs.append(input_)
			Guesses.append(guess_)
			ParticipantIDs.append(ID)
			Input_Types.append(row[15])
			Hidden_states.append(hmm_states)
			Accuracy.append(accuracy)
			Bonus_round.append(bonus_round)
		Inputs.pop(0)
		Guesses.pop(0)
		ParticipantIDs.pop(0)
		Input_Types.pop(0)
		Hidden_states.pop(0)
		Accuracy.pop(0)
		Bonus_round.pop(0)

In [4]:
Input_Types[1]

'NoisyPeriodic'

In [5]:
len(Inputs[1])

445

In [5]:
def rate(hidden_states,guesses):
	# get statistics
	# this only works when things are well-sampled
	joint_xy = {}
	marginal_x = {}
	marginal_y = {}
	for i in range(len(hidden_states)):
		xy = str(hidden_states[i])+str(guesses[i])
		x = str(hidden_states[i])
		y = str(guesses[i])
		if xy in joint_xy:
			joint_xy[xy] = joint_xy[xy]+1
		else:
			joint_xy[xy] = 1
		if x in marginal_x:
			marginal_x[x] = marginal_x[x]+1
		else:
			marginal_x[x] = 1
		if y in marginal_y:
			marginal_y[y] = marginal_y[y]+1
		else:
			marginal_y[y] = 1
	# hack to add appropriate pseudocounts
	all_cs = np.unique(hidden_states)
	all_output = [0,1]
	if len(marginal_y)<2:
		marginal_y[str(2)] = 0
	for cs in all_cs:
		for output in all_output:
			xy = str(cs)+str(output)
			if xy in joint_xy:
				pass
			else:
				joint_xy[xy] = 0
	# find tot
	tot_xy = np.sum(list(joint_xy.values()))
	tot_x = np.sum(list(marginal_x.values()))
	tot_y = np.sum(list(marginal_y.values()))
	# get the entropies
	p_xy = np.asarray(list(joint_xy.values()))/tot_xy
	H_xy = -np.nansum(p_xy*np.log2(p_xy))
	p_x = np.asarray(list(marginal_x.values()))/tot_x
	H_x = -np.nansum(p_x*np.log2(p_x))
	p_y = np.asarray(list(marginal_y.values()))/tot_y
	H_y = -np.nansum(p_y*np.log2(p_y))
	# get the mutual information
	I_xy = H_x+H_y-H_xy
	return I_xy

In [35]:
def ngram_learner(n,data_input,hidden_states):
	# first rate and accuracy is with no probability matching
	# second rate and accuracy is with probability matching
	data_input1 = []
	hidden_states1 = []
	guesses1 = []
	guesses2 = []
	for i in range(n+1,len(data_input)):
		# figure out what the probabilities are of guessing
		# last n symbols
		foo = data_input[i-n-1:i-1]
		foo0 = foo+'0'
		foo1 = foo+'1'
		p0 = data_input[:i].count(foo0)+1
		p1 = data_input[:i].count(foo1)+1
		guess1 = np.argmax([p0,p1])
		guess2 = np.random.choice(2,size=1,p=[p0/(p0+p1),p1/(p0+p1)])
		guesses1.append(guess1)
		guesses2.append(guess2)
		hidden_states1.append(hidden_states[i])
		data_input1.append(data_input[i])
	# get rates and accuracies
	M = len(guesses1); guesses1 = guesses1[int(M/2):M]
	guesses2 = guesses2[int(M/2):M]
	data_input1 = data_input1[int(M/2):M]; hidden_states1 = hidden_states1[int(M/2):M]
	Rate1 = rate(hidden_states1,guesses1)
	Rate2 = rate(hidden_states1,guesses2)
	Rates = [Rate1,Rate2]
	Acc1 = np.mean([int(data_input1[j])==guesses1[j] for j in range(int(M/2))])
	Acc2 = np.mean([int(data_input1[j])==guesses2[j] for j in range(int(M/2))]); Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [36]:
print(Input_Types[0])
print(bad_learners(Inputs[0]))
for i in range(10):
  print(ngram_learner(i,Inputs[0],Hidden_states[0]))

Clumpy
([0, 0], [0.5, np.float64(0.6140888208269525)])
([np.float64(0.0), np.float64(0.003995142400022189)], [np.float64(0.6165644171779141), np.float64(0.5153374233128835)])
([np.float64(0.40870523527940295), np.float64(0.006750528672093914)], [np.float64(0.39692307692307693), np.float64(0.48923076923076925)])
([np.float64(0.7030778604454733), np.float64(0.2462116542419519)], [np.float64(0.4276923076923077), np.float64(0.44)])
([np.float64(0.2269724174356451), np.float64(0.220874222653487)], [np.float64(0.43209876543209874), np.float64(0.45987654320987653)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.2930961844747073), np.float64(0.2174257926784695)], [np.float64(0.4228395061728395), np.float64(0.404320987654321)])
([np.float64(0.35529760603356864), np.float64(0.24158141756893858)], [np.float64(0.42724458204334365), np.float64(0.4520123839009288)])
([np.float64(0.3886291733917038), np.float64(0.20802643278166055)], [np.float64(0.4458204334365325), np.float64(0.43653250773993807)])
([np.float64(0.4478440795743024), np.float64(0.2431481142710723)], [np.float64(0.45962732919254656), np.float64(0.453416149068323)])
([np.float64(0.49812500039845675), np.float64(0.11872180062815163)], [np.float64(0.4782608695652174), np.float64(0.5031055900621118)])
([np.float64(0.4607799426421453), np.float64(0.14384724678920335)], [np.float64(0.4766355140186916), np.float64(0.42679127725856697)])


In [37]:
print(Input_Types[1])
print(bad_learners(Inputs[1]))
for i in range(10):
  print(ngram_learner(i,Inputs[1],Hidden_states[1]))

NoisyPeriodic
([0, 0], [0.5, np.float64(0.5460674157303371)])
([np.float64(0.0), np.float64(0.005944665286785833)], [np.float64(0.5315315315315315), np.float64(0.5495495495495496)])
([np.float64(0.8273863324079629), np.float64(0.6433939174900185)], [np.float64(0.06334841628959276), np.float64(0.09954751131221719)])
([np.float64(0.9999852306467514), np.float64(0.5943348075669299)], [np.float64(0.03167420814479638), np.float64(0.11764705882352941)])
([np.float64(0.9999852306467514), np.float64(0.5891172995422986)], [np.float64(0.031818181818181815), np.float64(0.11363636363636363)])
([np.float64(1.0), np.float64(0.5069515739843484)], [np.float64(0.031818181818181815), np.float64(0.1409090909090909)])
([np.float64(1.0), np.float64(0.44569467025931186)], [np.float64(0.0319634703196347), np.float64(0.1643835616438356)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.9999849596541528), np.float64(0.41346132892851917)], [np.float64(0.0319634703196347), np.float64(0.1917808219178082)])
([np.float64(0.9999849596541528), np.float64(0.4762531108502752)], [np.float64(0.03211009174311927), np.float64(0.15137614678899083)])
([np.float64(1.0), np.float64(0.31872816311598173)], [np.float64(0.03211009174311927), np.float64(0.22018348623853212)])
([np.float64(1.0), np.float64(0.42896699251827997)], [np.float64(0.03225806451612903), np.float64(0.17050691244239632)])


In [38]:
print(Input_Types[2])
print(bad_learners(Inputs[2]))
for i in range(10):
  print(ngram_learner(i,Inputs[2],Hidden_states[2]))

EvenProcess
([0, 0], [0.5, np.float64(0.5381062355658199)])
([np.float64(0.0), np.float64(0.00043427446810095915)], [np.float64(0.5462962962962963), np.float64(0.5138888888888888)])
([np.float64(0.014098873165398595), np.float64(0.003283621469629905)], [np.float64(0.4558139534883721), np.float64(0.4930232558139535)])
([np.float64(4.004802207613345e-05), np.float64(0.04338530190086609)], [np.float64(0.4930232558139535), np.float64(0.40930232558139534)])
([np.float64(0.013069192352290981), np.float64(0.0008264623218117162)], [np.float64(0.4392523364485981), np.float64(0.48598130841121495)])
([np.float64(0.020955699166470332), np.float64(0.02100073684826831)], [np.float64(0.4672897196261682), np.float64(0.46261682242990654)])
([np.float64(0.013437665652831887), np.float64(0.015317928745262499)], [np.float64(0.47417840375586856), np.float64(0.4460093896713615)])
([np.float64(0.009314987277197817), np.float64(0.008754820744268255)], [np.float64(0.47417840375586856), np.float64(0.51643192488

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.006469460033514229), np.float64(0.0068727622608180106)], [np.float64(0.47641509433962265), np.float64(0.49056603773584906)])
([np.float64(0.0013431353651460753), np.float64(3.00872209644254e-05)], [np.float64(0.5330188679245284), np.float64(0.5188679245283019)])
([np.float64(0.002545187797601045), np.float64(0.0081318223333402)], [np.float64(0.5308056872037915), np.float64(0.5592417061611374)])


In [32]:
def LSTM_learner(data_input,hidden_states):
	# first rate and accuracy is with no probability matching
	# second rate and accuracy is with probability matching
	timesteps = 5
	data_input1 = []
	hidden_states1 = []
	guesses_1 = [] # with no probability matching
	guesses_2 = [] # with probability matching
	tf.keras.backend.clear_session()
	# train an LSTM to predict in MSE sense
	batch_x = []; batch_y = []
	for i in range(timesteps,len(data_input)):
		batch_x.append([int(data_input[j]) for j in range(i-timesteps,i)])
		batch_y.append([int(data_input[i])])
	batch_x = np.asarray(batch_x)
	batch_y = np.asarray(batch_y)
	batch_x = batch_x.reshape((len(batch_x),timesteps,1))
	batch_y = batch_y.reshape((len(batch_y),1))
	batch_x = tf.convert_to_tensor(batch_x)
	batch_y = tf.convert_to_tensor(batch_y)
	#
	X = tf.keras.Input(name='X', shape=[timesteps, 1], dtype=tf.dtypes.float32)
	#
	lstm_output = tf.keras.layers.LSTM(50)(X)
	prediction = tf.keras.layers.Dense(1,activation='sigmoid')(lstm_output)
	opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
	model = keras.Model(inputs=X, outputs=prediction)
	model.compile(optimizer=opt, loss='binary_crossentropy')
	model.fit(x=batch_x, y=batch_y, epochs=100, verbose=0)
	for i in range(timesteps,len(data_input)):
		batch_x = [int(data_input[j]) for j in range(i-timesteps,i)]
		batch_y = [int(data_input[i])]
		batch_x = np.asarray(batch_x)
		batch_y = np.asarray(batch_y)
		batch_x = batch_x.reshape((1,timesteps,1))
		batch_y = batch_y.reshape((len(batch_y),1))
		batch_x = tf.convert_to_tensor(batch_x)
		batch_y = tf.convert_to_tensor(batch_y)
		xpreds = model.predict(batch_x)
		data_input1.append(int(data_input[i])); #print(batch_y.numpy()[-1])
		hidden_states1.append(hidden_states[i])
		guesses_1.append(np.round(xpreds[0][0]))
		p = np.array([1 - float(xpreds[0][0]), float(xpreds[0][0])], dtype=np.float64)
		guesses_2.append(np.random.choice(a=2,size=1,p=p/np.sum(p)))
	# get rates and accuracies
	M = len(guesses_1); guesses_1 = guesses_1[int(M/2):M]
	guesses_2 = guesses_2[int(M/2):M]
	data_input1 = data_input1[int(M/2):M]; hidden_states1 = hidden_states1[int(M/2):M]
	Rate1 = rate(hidden_states1,guesses_1)
	Rate2 = rate(hidden_states1,guesses_2)
	Rates = [Rate1,Rate2]
	Acc1 = np.mean([np.abs(data_input1[j]-guesses_1[j])<1e-5 for j in range(len(guesses_1))])
	Acc2 = np.mean([np.abs(data_input1[j]-guesses_2[j])<1e-5 for j in range(len(guesses_2))])
	Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [7]:
def bad_learners(data_input):
	foo = [int(i) for i in data_input]
	freq1 = np.mean(foo); freq0 = 1-freq1 # best guess
	Rate1 = 0
	Rate2 = 0
	Rates = [Rate1,Rate2]
	Acc1 = 0.5
	Acc2 = np.max([freq0,freq1])
	Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [33]:
r, a = LSTM_learner(Inputs[0],Hidden_states[0])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


In [34]:
print(len(Inputs[0])); print(len(Hidden_states[0]))

653
653


In [39]:
r, a = LSTM_learner(Inputs[1],Hidden_states[1])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))


In [40]:
r, a = LSTM_learner(Inputs[2],Hidden_states[2])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))
